In [1]:
from IPython.display import display, HTML, IFrame

In [2]:
from bootlets.templates import *

In [3]:
display(HTML(Header('Hello, World!').draw()))

In [15]:
html = Container(
            Header('Header 1', size=1),
            P('paragraph'),
            Div(P('seperate set of'), P('paragraphs', class_='bold')),
            UnorderedList(*['List', 'of', 'things']),
            DescriptionList({'title': 'description', 
                             'another title': 'another description',
                             'nested list': UnorderedList(*['one', 'two'])
                            })
        )

display(HTML(
    html.draw()
))

In [16]:
html

Container(Header('Header 1', size=1), Paragraph('paragraph'), Div(Paragraph('seperate set of'), Paragraph('paragraphs', class_='bold')), UnorderedList('List', 'of', 'things'), DescriptionList({'title': 'description', 'another title': 'another description', 'nested list': UnorderedList('one', 'two')}))

In [17]:
print(html.draw())

<h1>Header 1</h1>
<p>paragraph</p>
<div><p>seperate set of</p>
<p class="bold">paragraphs</p></div>
<ul><li>List</li>
<li>of</li>
<li>things</li></ul>
<dl><dt>title</dt><dd>description</dd>
<dt>another title</dt><dd>another description</dd>
<dt>nested list</dt><dd><ul><li>one</li>
<li>two</li></ul></dd></dl>
